In [1]:
%matplotlib inline                           
%config InlineBackend.figure_format='retina'       # Melhora a resolucao do plot
from IPython.core.display import display, HTML
import matplotlib.pylab as plt                     # Permite fazer plots utilizando o pacote matplotlib
plt.rcParams['figure.figsize'] = (9.0, 9.0)      # Determina o tamanho do plot dentro do ipython notebook
from astropy.io import fits
import numpy as np 
import pandas as pd
import math

**Motivação**: Esse código foi criado apenas após meses de trabalho no projeto pois percebi a necessidade de preservar em apenas uma pasta todas as modificações realizadas no catálogo. Até então, eu havia adicionado, corrigido e manipulado os parâmetros em pastas diferentes. Isso gera uma confusão na hora de saber qual é o catálogo mais atual e uma mudança frequente de nomes para cada nova versão. Portanto compilei todas as modificações em um único notebook, gerando ao final de cada rodada apenas um arquivo de texto com nome único.

**Modificações realizadas anteriormente:** Listo brevemente as modificações realizadas até então. 

* Adição dos quatro catálogos disponíveis: Blakeslee, Demarco, Patel e Tanaka.
* Realizado uma união de todos os catálogos baseados nas distâncias angulares.
* Adicionando para os objetos membros a magnitude U do VMOS

# 1 - Adicionando catálogos

## 1.1 - Adicionando catálogo Blakeslee

* Artigo original: Blakeslee et al., 2006

Descrição:

| Nome | Parâmetro | Unidade|
| :---         |     :---      |          :--- |          :--- |
| ID_B              | Identificação     | --    |
| RA_B              | Ascensão reta     | graus (J2000)    |
| DEC_B             | Declinação        | graus (J2000)     |
| F775W_B           | magnitude i SEX   | mag    |
| F775W_GALFIT_T    | magnitude i GALFIT| mag    |
| Re_T              | Raio efetivo      | arcsec    |
| nSer_T            | índice de Sérsic  | --    |
| Morph_T           | Tipo T (1)            | --    |
| ba_T              | razão de eixos    | --    |
| B_T               | Bumpiness         | --    |
| Sigma_T           | Densidade superficial de massa (2)| --    |
| r_i_T             | r - i     | mag    |
| e_r_i_T           | erro r - i     | mag    |
| i_z_T             | i - z     | mag    |
| e_i_z_T           | erro i - z     | mag    |
| r_z_T             | r - z     | mag    |
| e_r_z_T           | erro r - z     | mag    |

(1) Postman+2005

In [4]:
#Adicionando catalogos
cat_blakeslee = np.loadtxt('../00-Catalogos/catalog_blakeslee.txt')
cat_blakeslee = pd.DataFrame(cat_blakeslee)

#Adicionando o nome de cada coluna
cat_blakeslee.columns = ['ID_B','RA_B','DEC_B','F775W_B','F775W_GALFIT_B','Re_B','nSer_B','Morph_B','ba_B','B_B','sigma_B','r_i_B','e_r_i_B','i_z_B','e_i_z_B','r_z_B','e_r_z_B']
#cat_blakeslee

## 1.2 -  Adicionando catalogo Tanaka 

* Artigo original: Tanaka et al., 2006 (https://arxiv.org/pdf/astro-ph/0511296.pdf)

Descrição:


| Nome | Parâmetro | Unidade|
| :---         |     :---      |          :--- |      
| nm_col_T     | Numero coluna Tanaka            | --    |
| ID_T         | Identificação                   |  --   |
| RA_T         | Ascensão Reta                   | graus (J2000) |
| DEC_T        | Declinação                      | graus (J2000) |
| zmag_T       | z                               | mag    |
| Rci_T        | R - i                           | mag    |
| i_z_T        | i - z                           | mag    |
| zph_T        | Redshift fotometrico            | --     |
| zsp_T        | Redshift Espectroscópico        | --     |
| Zspmin_T     | Redshift Espectroscópico mínimo | --     |
| zpsmax_T     | Redshift Espectroscópico máximo | --     |
| zconf_T      | Confiabilidade do redshift      | --     |


In [6]:
#Adicionando catálogo Tanaka:

cat_tanaka = np.loadtxt('../00-Catalogos/catalog_tanaka.txt', dtype = 'string')
cat_tanaka = pd.DataFrame(cat_tanaka)

#Transformando as coordenadas de horas -> graus
a = ((((pd.to_numeric(cat_tanaka[4])/60) + pd.to_numeric(cat_tanaka[3]))/60) + pd.to_numeric(cat_tanaka[2]))*15
b = ((((pd.to_numeric(cat_tanaka[7])/60) + pd.to_numeric(cat_tanaka[6]))/60) + abs(pd.to_numeric(cat_tanaka[5])))*(-1)
cat_tanaka.insert(2,'2',a)
del cat_tanaka[2]
del cat_tanaka[3]
del cat_tanaka[4]

cat_tanaka.insert(3,'3',b)
del cat_tanaka[5]
del cat_tanaka[6]
del cat_tanaka[7]
#print cat_tanaka
cat_tanaka.columns = ['nm_col_T','ID_T','RA_T','DEC_T','zmag_T','VRc_T','Rci_T','i_z_T','zph_T','zsp_T','Zspmin_T','zpsmax_T','zconf_T']

#cat_tanaka

## 1.3 - Adicionando catálogo Demarco

* Artigo original: Demarco et al., 2010



| Nome | Parâmetro | Unidade|
| :---         |     :---      |          :--- |
|ID_D|  Identificão     | --    |
|RA_D|  Ascensão Reta     |  graus   |
|DEC_D|  Declinação     |  graus   |
|Z_D|  Redshift     | --    |
|EL_D| Flag de linhas de emissão      |  --   |
|TYPE_D|  Tipo T     |  --   |
|AUTO_R_D|  Magnitude R abertura mag_auto     |  mag   |
|ERR_AUTO_R_D|   Erro R MAG_AUTO   |  mag   |
|AP_R_D|   Magnitude R abertura 2arc    | mag    |
|ERR_AP_R_D|   Erro R 2arc     | mag    |
|AUTO_I_D|  Magnitude I abertura mag_auto     |  mag   |
|ERR_AUTO_I_D|   Erro I MAG_AUTO     | mag    |
|AP_I_D|   Magnitude I abertura 2arc    |  mag   |
|ERR_AP_I_D|   Erro I 2arc    |  mag   |
|AUTO_Z_D|  Magnitude Z abertura mag_auto     | mag    |
|ERR_AUTO_Z_D|   Erro Z MAG_AUTO     |  mag   |
|AP_Z_D|  Magnitude Z abertura 2arc     |  mag   |
|ERR_AP_Z_D|   Erro Z 2arc    | mag    |
|AUTO_J_D|  Magnitude J abertura mag_auto     | mag    |
|ERR_AUTO_J_D|   Erro J MAG_AUTO     | mag    |
|AP_J_D|  Magnitude J abertura 2arc     | mag    |
|ERR_AP_J_D|   Erro J 2arc    | mag    |
|AUTO_KS_D|  Magnitude Ks abertura mag_auto     |  mag   |
|ERR_AUTO_KS_D| Erro Ks MAG_AUTO       | mag    |
|AP_KS_D|  Magnitude Ks abertura 2arc     | mag    |
|ERR_AP_KS_D|   Erro Ks 2arc    |mag     |
|STEL_MASS_D|  Massa estelar     |     |
       

In [9]:
cat_demarco = np.loadtxt('../00-Catalogos/catalog_demarco.txt', dtype = 'string')
cat_demarco = pd.DataFrame(cat_demarco)

cat_demarco.columns =['ID_D','RA_D','DEC_D','Z_D','EL_D', 'TYPE_D' ,'AUTO_R_D' ,'ERR_AUTO_R_D', 'AP_R_D'   ,'ERR_AP_R_D' ,'AUTO_I_D' ,'ERR_AUTO_I_D' ,'AP_I_D',   'ERR_AP_I_D', 'AUTO_Z_D', 'ERR_AUTO_Z_D', 'AP_Z_D',   'ERR_AP_Z_D', 'AUTO_J_D', 'ERR_AUTO_J_D', 'AP_J_D',   'ERR_AP_J_D', 'AUTO_KS_D', 'ERR_AUTO_KS_D', 'AP_KS_D',  'ERR_AP_KS_D', 'STEL_MASS_D']


## 1.4 - Adicionando Patel

*  Artigo: Patel+2009 (http://iopscience.iop.org/article/10.1088/0004-637X/694/2/1349/pdf)

Descrição:

| Nome | Parâmetro | Unidade|
| :---         |     :---      |          :--- |     
| ID |  Identificação |  --   |      
| RA  |  Ascensão Reta  |  graus   |             
| DEC  | Declinação   | graus    |                 
| zmag_auto| Magnitude z mag_auto  |  mag   |           
| zmag_col  | Magnitude z ap 2arc   |  mag   |         
| Ks_col| Magnitude Ks ap 2arc   |  mag   |             
| z50    |    |  --   |               
| z025|    |  --   |      
|z975_P|    |  --   |      
|zspec_P|    | --    |      
|lmass_P|    |     |      
|density_P|   |     |      

In [11]:
cat_patel = np.loadtxt('../Catalogos/catalog_patel.txt', dtype = 'string')
cat_patel = pd.DataFrame(cat_patel)
cat_patel.columns = ['ID_P','RA_P','DEC_P','zmag_auto_P','zmag_col_P','Ks_col_P','z50_P','z025_z_P','z975_P','zspec_P','lmass_P','density_P']
